First we define our basic data types and some helper functions

In [3]:
let pp_approx fmt r =
    CCFormat.fprintf fmt "%s" (Real.to_string_approx r) [@@program]

#install_printer pp_approx

val pp_approx : CCFormat.t -> Q.t -> unit = <fun>


In [4]:
type pos = {x : int; y: int};;

type move = Up | Down | Left | Right;;

let move2coord move = 
    if move = Up then (0, 1) else
    if move = Down then (0, -1) else
    if move = Left then (-1, 0) else
    (1, 0);;

let new_pos move pos = 
    let (c1, c2) = move2coord move in
    {x = pos.x + c1;
    y = pos.y + c2};;
    
let l1_dist pos1 pos2 = abs(pos1.x - pos2.x) + abs(pos1.y - pos2.y);;

type pos = { x : int; y : int; }
type move = Up | Down | Left | Right
val move2coord : move -> Z.t * Z.t = <fun>
val new_pos : move -> pos -> pos = <fun>
val l1_dist : pos -> pos -> Z.t = <fun>


Then we need our transition function and a policy

In [5]:
let trans p1 m p2 =
if l1_dist (new_pos m p1) p2 = 0 then 0.8 else
if l1_dist (new_pos m p1) p2 = 1 then 0.1 else
0.2;;

let pi s a =
    if s.x mod 2 = 0 then
        if a = Up then 0.7 else 0.1
    else
        if a = Right then 0.7 else 0.1;;
        
(* { c with car_speed = c.car_max_speed } *)

val trans : pos -> move -> pos -> Q.t = <fun>
val pi : pos -> move -> Q.t = <fun>


Different stuff

In [6]:
let base x y =
    if 0.0 <=. x && x <=. 1.0
    && 0.0 <=. y && y <=. 1.0
    then true else false;;
    
type age = Young | Medium | Old;;


type person = {is_female : bool;
               age       : age};;
    
let generator (x, y) = 
    let f =
        if x <=. 0.45 then
            true
        else
            false
    in let a =
        if y <=. 0.3 then
            Young
        else if y <=. 0.7 then
            Medium
        else
            Old
    in {is_female = f;
        age = a};;
        
let gets_job p =
    if (p.age = Medium && not p.is_female) then "gets_job" else "nope";;
    
let model x y = generator (x, y) |> gets_job;;

let rs = Decompose.top ~assuming:"base" "model" [@@program];;

val base : real -> real -> bool = <fun>
type age = Young | Medium | Old
type person = { is_female : bool; age : age; }
val generator : real * real -> person = <fun>
val gets_job : person -> string = <fun>
val model : real -> real -> string = <fun>
val rs : Imandra_interactive.Decompose.t list =
  [<region>; <region>; <region>; <region>]


Constraints,Invariant
not (<=. y 3/10)not (<=. y 7/10),"""nope"""
not (<=. y 3/10)<=. y 7/10<=. x 9/20,"""nope"""
not (<=. y 3/10)<=. y 7/10not (<=. x 9/20),"""gets_job"""
<=. y 3/10,"""nope"""


In [7]:
let c = List.hd((List.hd(List.tl(rs))).reg_constraints) [@@program];;

let d = Term.to_doc(c) [@@program];;


val c : Imandra_surface.Term.t = not (<=. y 3/10)
val d : Imandra_surface.Document.t = <abstr>


Define some helper functions for approximating useful distributions using Taylor series'

In [8]:
let power_measure x =
 if x < 0 then abs x + 1
 else x

let rec power (x : real) (y : int) =
    if y = 0 then 1.0 
    else if y > 0 then x *. power x (y - 1)
    else 1.0 /. (power x (-y))
[@@measure Ordinal.of_int (power_measure y)]

let rec factorial x =
    if x <= 1 then 1 else x * factorial (x - 1)
    
let rec ln (x : real) (k : int) = 
    if k <= 1 then 
        (x -. 1.0) /. Real.of_int(k)
    else
        ((power (-1.0) (k - 1)) *. (power (x -. 1.0) k) /. Real.of_int(k)) +. ln x (k - 1);;
        
let ln_approx x = ln x 5;;

let rec exp (x : real) (k : int) = 
    if k <= 0 then 
        1.0
    else
        ((power x k) /. Real.of_int((factorial k))) +. (exp x (k-1));;
        
let exp_approx x = exp x 30;;

val power_measure : int -> int = <fun>
val power : real -> int -> real = <fun>
val factorial : int -> Z.t = <fun>
val ln : real -> int -> real = <fun>
val ln_approx : real -> real = <fun>
val exp : real -> int -> real = <fun>
val exp_approx : real -> real = <fun>


termination proof Termination proof call `power x (y - 1)` from `power x y` original power x y sub power x (y - 1) original ordinal Ordinal.Int (if power_measure y >= 0 then power_measure y else 0) sub ordinal Ordinal.Int (if power_measure (y - 1) >= 0 then power_measure (y - 1) else 0) path [y > 0 && not (y = 0)] proof detailed proof summary full ground_instances 1 definitions 0 inductions 0 search_time 0.040s details Expand smt_stats num checks 3 arith assert lower 13 arith pivots 4 rlimit count 3055 mk clause 24 datatype occurs check 2 mk bool var 39 arith assert upper 5 decisions 7 arith add rows 6 arith bound prop 2 propagations 11 conflicts 3 arith fixed eqs 4 datatype accessor ax 2 arith conflicts 1 arith assert diseq 3 num allocs 3702627105 final checks 1 added eqs 11 del clause 15 arith eq adapter 7 memory 39.450000 max memory 91.980000 Expand start[0.040s]
 y > 0
 && not (y = 0)
 && (if (if y < 0 then (if y >= 0 then y else …) + 1 else y) >= 0
 then if y < 0 then (if y >= 0 then y else …) + 1 else y else 0)
 >= 0
 && (if (if (y - 1) < 0 then (if (y - 1) >= 0 then y - 1 else …) + 1
 else (y - 1))
 >= 0
 then
 if (y - 1) < 0 then (if (y - 1) >= 0 then y - 1 else …) + 1
 else y - 1
 else 0)
 >= 0
 ==> not ((y - 1) > 0 && not (y - 1 = 0))
 && not (not ((y - 1) > 0) && not (y - 1 = 0))
 || Ordinal.<<
 (Ordinal.Int
 (if (if (y - 1) < 0 then (if (y - 1) >= 0 then y - 1 else …) + 1
 else (y - 1))
 >= 0
 then
 if (y - 1) < 0 then (if (y - 1) >= 0 then y - 1 else …) + 1
 else y - 1
 else 0))
 (Ordinal.Int
 (if (if y < 0 then (if y >= 0 then y else …) + 1 else y) >= 0
 then if y < 0 then (if y >= 0 then y else …) + 1 else y 
 else 0)) simplify into (not
 (((not (y <= 0) && not (y = 0))
 && (if (if 0 <= y then y else (1 + (if y >= 0 then y else -1 * y))) >= 0
 then if 0 <= y then y else 1 + (if y >= 0 then y else -1 * y) 
 else 0)
 >= 0)
 && (if (if 1 <= y then -1 + y
 else (1 + (if y >= 1 then -1 + y else 1 + -1 * y)))
 >= 0
 then
 if 1 <= y then -1 + y
 else 1 + (if y >= 1 then -1 + y else 1 + -1 * y)
 else 0)
 >= 0)
 || not (not (y <= 1) && not (y = 1)) && not (y <= 1 && not (y = 1)))
|| Ordinal.<<
 (Ordinal.Int
 (if (if 1 <= y then -1 + y
 else (1 + (if y >= 1 then -1 + y else 1 + -1 * y)))
 >= 0
 then
 if 1 <= y then -1 + y else 1 + (if y >= 1 then -1 + y else 1 + -1 * y)
 else 0))
 (Ordinal.Int
 (if (if 0 <= y then y else (1 + (if y >= 0 then y else -1 * y))) >= 0
 then if 0 <= y then y else 1 + (if y >= 0 then y else -1 * y) else 0)) expansions [] rewrite_steps forward_chaining unroll expr (let ((a!1 (+ 1 (ite (>= y_1564 1) (+ (- 1) y_1564) (+ 1 (* (- 1) y_1564)))))
 (a!3 (ite (<= 0 … expansions unsat (let ((a!1 (+ 1 (ite (>= y_1564 0) y_1564 (* (- 1) y_1564))))
 (a!10 (+ 1 (ite (>= y_1564 1) (+… call `power x (~- y)` from `power x y` original power x y sub power x (~- y) original ordinal Ordinal.Int (if power_measure y >= 0 then power_measure y else 0) sub ordinal Ordinal.Int (if power_measure (~- y) >= 0 then power_measure (~- y) else 0) path [not (y > 0) && not (y = 0)] proof detailed proof summary full ground_instances 1 definitions 0 inductions 0 search_time 0.013s details Expand smt_stats num checks 3 arith assert lower 7 arith pivots 4 rlimit count 1382 mk clause 9 datatype occurs check 2 mk bool var 28 arith assert upper 7 decisions 4 arith add rows 7 arith bound prop 1 propagations 4 conflicts 2 arith fixed eqs 4 datatype accessor ax 2 arith conflicts 1 arith assert diseq 1 num allocs 2771460918 final checks 1 added eqs 10 del clause 9 arith eq adapter 5 memory 89.000000 max memory 89.000000 Expand start[0.013s]
 not (y > 0)
 && not (y = 0)
 && (if (if y < 0 then (if y >= 0 then y else …) + 1 else y) >= 0
 then if y < 0 then (if y >= 0 then y else …) + 1 else y else 0)
 >= 0
 && (if (if … < 0 then (if … >= 0 then … else …) + 1 else …)
 >= 0
 then if … < 0 then (if … >= 0 then … else …) + 1 else …
 else 0)
 >= 0
 ==> not (… > 0 && not (… = 0)) && not (not (… > 0) && not (… = 0))
 || Ordinal.<<
 (Ordinal.Int
 (if (

termination proof Termination proof call `factorial (x - 1)` from `factorial x` original factorial x sub factorial (x - 1) original ordinal Ordinal.Int (if x >= 0 then x else 0) sub ordinal Ordinal.Int (if (x - 1) >= 0 then x - 1 else 0) path [not (x <= 1)] proof detailed proof summary full ground_instances 1 definitions 0 inductions 0 search_time 0.011s details Expand smt_stats num checks 3 arith assert lower 8 arith pivots 2 rlimit count 1027 mk clause 5 datatype occurs check 2 mk bool var 20 arith assert upper 3 decisions 2 arith add rows 3 propagations 2 conflicts 2 arith fixed eqs 2 datatype accessor ax 2 arith conflicts 1 num allocs 3772628995 final checks 1 added eqs 6 del clause 5 arith eq adapter 2 memory 42.780000 max memory 91.980000 Expand start[0.011s]
 not (x <= 1)
 && (if x >= 0 then x else 0) >= 0
 && (if (x - 1) >= 0 then x - 1 else 0) >= 0
 ==> (x - 1) <= 1
 || Ordinal.<< (Ordinal.Int (if (x - 1) >= 0 then x - 1 else 0))
 (Ordinal.Int (if x >= 0 then x else 0)) simplify into (not
 ((not (x <= 1) && (if x >= 0 then x else 0) >= 0)
 && (if x >= 1 then -1 + x else 0) >= 0)
 || x <= 2)
|| Ordinal.<< (Ordinal.Int (if x >= 1 then -1 + x else 0))
 (Ordinal.Int (if x >= 0 then x else 0)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_116| (|Ordinal.Int_107| (ite (>= x_1580 1) (+ (- 1) x_1580) 0))
 (|Ord… expansions unsat (let ((a!1 (not (= x_1580 (ite (>= x_1580 0) x_1580 0))))
 (a!2 (+ x_1580 (* (- 1) (ite (>= x_1…

termination proof Termination proof call `ln x (k - 1)` from `ln x k` original ln x k sub ln x (k - 1) original ordinal Ordinal.Int (if k >= 0 then k else 0) sub ordinal Ordinal.Int (if (k - 1) >= 0 then k - 1 else 0) path [not (k <= 1)] proof detailed proof summary full ground_instances 1 definitions 0 inductions 0 search_time 0.011s details Expand smt_stats num checks 3 arith assert lower 8 arith pivots 2 rlimit count 1027 mk clause 5 datatype occurs check 2 mk bool var 20 arith assert upper 3 decisions 2 arith add rows 3 propagations 2 conflicts 2 arith fixed eqs 2 datatype accessor ax 2 arith conflicts 1 num allocs 3886554479 final checks 1 added eqs 6 del clause 5 arith eq adapter 2 memory 43.170000 max memory 91.980000 Expand start[0.011s]
 not (k <= 1)
 && (if k >= 0 then k else 0) >= 0
 && (if (k - 1) >= 0 then k - 1 else 0) >= 0
 ==> (k - 1) <= 1
 || Ordinal.<< (Ordinal.Int (if (k - 1) >= 0 then k - 1 else 0))
 (Ordinal.Int (if k >= 0 then k else 0)) simplify into (not
 ((not (k <= 1) && (if k >= 0 then k else 0) >= 0)
 && (if k >= 1 then -1 + k else 0) >= 0)
 || k <= 2)
|| Ordinal.<< (Ordinal.Int (if k >= 1 then -1 + k else 0))
 (Ordinal.Int (if k >= 0 then k else 0)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_116| (|Ordinal.Int_107| (ite (>= k_1593 1) (+ (- 1) k_1593) 0))
 (|Ord… expansions unsat (let ((a!1 (not (= k_1593 (ite (>= k_1593 0) k_1593 0))))
 (a!2 (+ k_1593 (* (- 1) (ite (>= k_1…

termination proof Termination proof call `exp x (k - 1)` from `exp x k` original exp x k sub exp x (k - 1) original ordinal Ordinal.Int (if k >= 0 then k else 0) sub ordinal Ordinal.Int (if (k - 1) >= 0 then k - 1 else 0) path [not (k <= 0)] proof detailed proof summary full ground_instances 1 definitions 0 inductions 0 search_time 0.010s details Expand smt_stats num checks 3 arith assert lower 8 arith pivots 2 rlimit count 1027 mk clause 4 datatype occurs check 2 mk bool var 20 arith assert upper 3 decisions 2 arith add rows 3 propagations 2 conflicts 2 arith fixed eqs 2 datatype accessor ax 2 arith conflicts 1 num allocs 4005622919 final checks 1 added eqs 6 del clause 4 arith eq adapter 2 memory 43.710000 max memory 91.980000 Expand start[0.010s]
 not (k <= 0)
 && (if k >= 0 then k else 0) >= 0
 && (if (k - 1) >= 0 then k - 1 else 0) >= 0
 ==> (k - 1) <= 0
 || Ordinal.<< (Ordinal.Int (if (k - 1) >= 0 then k - 1 else 0))
 (Ordinal.Int (if k >= 0 then k else 0)) simplify into (not
 ((not (k <= 0) && (if k >= 0 then k else 0) >= 0)
 && (if k >= 1 then -1 + k else 0) >= 0)
 || k <= 1)
|| Ordinal.<< (Ordinal.Int (if k >= 1 then -1 + k else 0))
 (Ordinal.Int (if k >= 0 then k else 0)) expansions [] rewrite_steps forward_chaining unroll expr (|Ordinal.<<_116| (|Ordinal.Int_107| (ite (>= k_1608 1) (+ (- 1) k_1608) 0))
 (|Ord… expansions unsat (let ((a!1 (not (= k_1608 (ite (>= k_1608 0) k_1608 0))))
 (a!2 (+ k_1608 (* (- 1) (ite (>= k_1…

Use quantile functions to define generators that map the unit hypercube over the reals to stochastic outcomes. Normal quantile function approximated using the formula from "Very Simply Explicitly Invertible Approximations of
Normal Cumulative and Normal Quantile Function".

In [9]:
let bernoulli (x : real) (p : real) = if x <. p then true else false;;

(* let f x = bernoulli x 0.7;;

f 0.9;; *)

let rec categorical (x : real) (y : real) (c : string list) (p : real list) =
    match c, p with
        | [], _ -> "none"
        | _, [] -> "none"
        | cls :: cs, prb :: ps -> 
            if x <=. prb +. y then cls
            else categorical x (prb +. y) cs ps
[@@measure Ordinal.of_int (List.length c)];;

(* let f x = categorical x 0.0 ["a"; "b"; "c"] [0.3; 0.45; 0.25];;

f 0.5;; *)

let uniform (x : real) (a : real) (b : real) = a +. ((b -. a) *. x);;

(* let f x = uniform x 5.0 8.0;;

f 0.5;;
 *)
 
let exponential (x : real) (lambda : real) = -.(ln_approx (1.0 -. x)) /. lambda;;

(* let f x = exponential x 1.0;;

f 0.5;; *)

let logistic (x : real) (mu : real) (s : real) = mu +. (s *. ln_approx (x /. (1.0 -. x)));;

(* let f x = logistic x 1.0 0.5;;

f 0.5;; *)

let normal (x : real) (mu : real) (sigma : real) = let open Real in
    let a = ln_approx 2.0 in
    let b = ln_approx 22.0 in
    let c = ln_approx 41.0 in
    if x <= 0.5 then
        (10.0 / c) * ln_approx (1.0 - (ln_approx((- ln_approx x) / a) / b))
    else
        let y = 1.0 - x in
        - (10.0 / c) * ln_approx (1.0 - (ln_approx((- ln_approx y) / a) / b));;
        
let f x = logistic x 1.0 0.5;;

f 0.75;;


val bernoulli : real -> real -> bool = <fun>
val categorical : real -> real -> string list -> real list -> string = <fun>
val uniform : real -> real -> real -> real = <fun>
val exponential : real -> real -> real = <fun>
val logistic : real -> real -> real -> real = <fun>
val normal : real -> real -> real -> real = <fun>
val f : real -> real = <fun>
- : real = 3.53333333333


termination proof Termination proof call `categorical x (+. (List.hd p) y) (List.tl c) (List.tl p)` from `categorical x y c p` original categorical x y c p sub categorical x (+. (List.hd p) y) (List.tl c) (List.tl p) original ordinal Ordinal.Int (if List.length c >= 0 then List.length c else 0) sub ordinal Ordinal.Int
(if List.length (List.tl c) >= 0 then List.length (List.tl c) else 0) path [not (<=. x (+. (List.hd p) y)) && not (p = []) && not (c = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.017s details Expand smt_stats num checks 7 arith assert lower 7 arith pivots 2 rlimit count 2229 mk clause 6 datatype occurs check 40 mk bool var 67 arith assert upper 3 datatype splits 8 decisions 17 arith add rows 1 seq num reductions 5 propagations 4 interface eqs 3 conflicts 10 datatype accessor ax 8 arith conflicts 1 datatype constructor ax 18 seq extensionality 3 num allocs 4135732032 final checks 11 added eqs 53 del clause 3 arith eq adapter 2 memory 44.640000 max memory 91.980000 Expand start[0.017s]
 not (<=. x (+. (List.hd p) y))
 && not (p = [])
 && not (c = [])
 && (if List.length c >= 0 then List.length c else 0) >= 0
 && (if List.length (List.tl c) >= 0 then List.length (List.tl c)
 else 0)
 >= 0
 ==> not
 (not (<=. x (+. (List.hd (List.tl p)) (+. (List.hd p) y)))
 && not (List.tl p = []) && not (List.tl c = []))
 || Ordinal.<<
 (Ordinal.Int
 (if List.length (List.tl c) >= 0 then List.length (List.tl c)
 else 0))
 (Ordinal.Int (if List.length c >= 0 then List.length c else 0)) simplify into (not
 ((((not (<=. x (+. (List.hd p) y)) && not (p = [])) && not (c = []))
 && (if List.length c >= 0 then List.length c else 0) >= 0)
 && (if List.length (List.tl c) >= 0 then List.length (List.tl c) else 0) >=
 0)
 || not
 ((not (<=. x (+. (+. (List.hd (List.tl p)) (List.hd p)) y))
 && not (List.tl p = []))
 && not (List.tl c = [])))
|| Ordinal.<<
 (Ordinal.Int
 (if List.length (List.tl c) >= 0 then List.length (List.tl c) else 0))
 (Ordinal.Int (if List.length c >= 0 then List.length c else 0)) expansions [] rewrite_steps forward_chaining unroll expr (let ((a!1 (ite (>= (|List.length_1630| (|get.::.1_1625| c_1642)) 0)
 (|List.length_1… expansions unroll expr (|List.length_1630| (|get.::.1_1625| c_1642)) expansions unroll expr (|List.length_1630| c_1642) expansions unsat (let ((a!1 (ite (>= (|List.length_1630| (|get.::.1_1625| c_1642)) 0)
 (|List.length_1…

In [10]:
let base x y =
    if 0.0 <=. x && x <=. 1.0
    && 0.0 <=. y && y <=. 1.0
    then true else false;;
    
type age = Young | Medium | Old;;


type person = {gender    : string;
               age       : real};;
    
let generator (x, y) = 

    let f = categorical x 0.0 ["woman"; "man"; "other"] [0.3; 0.45; 0.25] in
    let a = normal y 30.0 5.0

    in {gender = f;
        age = a};;
        
let gets_job p =
    if (p.age <=. 45.0 && p.gender = "woman") then "gets_job" else "nope";;
    
let model x y = generator (x, y) |> gets_job;;

Decompose.top ~basis:["ln_approx"] ~assuming:"base" "model";;

val base : real -> real -> bool = <fun>
type age = Young | Medium | Old
type person = { gender : string; age : real; }
val generator : real * real -> person = <fun>
val gets_job : person -> string = <fun>
val model : real -> real -> string = <fun>
- : Imandra_interactive.Decompose.t list =
[<region>; <region>; <region>; <region>; <region>; <region>]


Constraints,Invariant
not (<=. y 1/2)not (>=. (*. (ln_approx (+. 1 (*. -1 (/. (ln_approx (*. -1 (/. (ln_approx (+. 1 (*. -1 y))) (ln_approx 2)))) (ln_approx 22))))) (/. 10 (ln_approx 41))) -45),"""nope"""
not (<=. y 1/2)>=. (*. (ln_approx (+. 1 (*. -1 (/. (ln_approx (*. -1 (/. (ln_approx (+. 1 (*. -1 y))) (ln_approx 2)))) (ln_approx 22))))) (/. 10 (ln_approx 41))) -45not (<=. x 3/10),"""nope"""
not (<=. y 1/2)>=. (*. (ln_approx (+. 1 (*. -1 (/. (ln_approx (*. -1 (/. (ln_approx (+. 1 (*. -1 y))) (ln_approx 2)))) (ln_approx 22))))) (/. 10 (ln_approx 41))) -45<=. x 3/10,"""gets_job"""
<=. y 1/2not (<=. (*. (ln_approx (+. 1 (*. -1 (/. (ln_approx (*. -1 (/. (ln_approx y) (ln_approx 2)))) (ln_approx 22))))) (/. 10 (ln_approx 41))) 45),"""nope"""
<=. y 1/2<=. (*. (ln_approx (+. 1 (*. -1 (/. (ln_approx (*. -1 (/. (ln_approx y) (ln_approx 2)))) (ln_approx 22))))) (/. 10 (ln_approx 41))) 45not (<=. x 3/10),"""nope"""
<=. y 1/2<=. (*. (ln_approx (+. 1 (*. -1 (/. (ln_approx (*. -1 (/. (ln_approx y) (ln_approx 2)))) (ln_approx 22))))) (/. 10 (ln_approx 41))) 45<=. x 3/10,"""gets_job"""
